In [1]:
import pandas as pd
import numpy as np
import requests
import data_getter as dg
import time
import os
# pd.options.display.max_rows = 999
# pd.reset_option("display.max_rows")
pd.get_option("display.max_columns")
pd.set_option("display.max_columns", 999)

In [2]:
def print_df_info_and_sample(df):
    print(df.info())
    print("========================================")
    print(df.head())

In [3]:
boxscore_list_df = dg.get_boxscore_list_to_df()

In [4]:
# print(boxscore_list_df.info())
# print("========================================")
# print(boxscore_list_df.head())
# print_df_info_and_sample(boxscore_list_df)
# list(boxscore_list_df.GAME_ID.unique())

In [5]:
game_id_list = list(boxscore_list_df.GAME_ID.unique())

In [6]:
len(game_id_list)

700

In [16]:
# test_pp_df = dg.get_one_game_play_to_play_to_df(game_id='0021700130')
# test_pp_df[['HOMEDESCRIPTION', 'NEUTRALDESCRIPTION','VISITORDESCRIPTION']].info()
# print_df_info_and_sample(test_pp_df)
# pp_game_file = "pp_"+"0021700130"+".csv"
# test_pp_df.to_csv("data/"+pp_game_file,header=True,index=False)

In [7]:
pp_df_dict = {}
for i in game_id_list:
    filename = "data/pp/"+"pp_"+str(i)+".csv"
    if os.path.isfile(filename):
        pp_df = pd.read_csv(filename)
    else:   
        pp_df = dg.get_one_game_play_to_play_to_df(game_id=i) 
        pp_df.to_csv("data/pp/"+"pp_"+str(i)+".csv", header=True, index=False)
    pp_df_dict[i] = pp_df
    time.sleep(0.5)

In [103]:
games_count = len(pp_df_dict)

### Dump data into pickle

In [46]:
import pickle
import datetime

In [48]:
date_str = datetime.datetime.today().strftime('%Y%m%d')
print(date_str)
pp_dump_file_name = "parm_and_plays_list_"+str(date_str)+".p"
print(os.path.join('data', pp_dump_file_name))

20171118
data/parm_and_plays_list_20171118.p


In [49]:
pickle.dump((game_id_list, pp_df_dict), open(os.path.join('data', pp_dump_file_name), 'wb'))

### Check point

In [54]:
import pickle
game_id_list, pp_df_dict = pickle.load(open('data/parm_and_plays_list_20171118.p', 'rb'))

In [81]:
main_column_list = ['PLAYER1_NAME', 'PLAYER2_NAME', 'PLAYER3_NAME', 'HOMEDESCRIPTION', 'NEUTRALDESCRIPTION',
       'VISITORDESCRIPTION']

In [89]:
test_df_dict = {}

for i in range(10):
    sel_id = "002170013" + str(i)
    test_df_dict[sel_id] = pp_df_dict[sel_id]



In [100]:
description_text = ""
for game_id, game_df in pp_df_dict.items():
# for game_id, game_df in test_df_dict.items():
    # game_pp_text = dg.get_one_pp_to_text(game_df)
    game_pp_text = get_one_pp_to_text(game_df)
    start_text = "GAME_START\n"
    end_text = "GAME_OVER"
    full_game_pp_text = start_text + game_pp_text + end_text
    # full_game_pp_text = "\n".join([start_text, game_pp_text, end_text])
    description_text = "\n\n".join([description_text, full_game_pp_text])

In [104]:
games_count = len(pp_df_dict)
pp_description_text_file_name = "games_count_and_description_text.p"
pickle.dump((games_count, description_text), open(os.path.join('data', pp_description_text_file_name), 'wb'))

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [101]:
text = description_text
view_play_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
games = text.split('\n\n')
print('Number of games: {}'.format(len(games)))
play_count_game = [game.count('\n') for game in games]
print('Average number of games in each game: {}'.format(np.average(play_count_game)))

plays = [play for game in games for play in game.split('\n')]
print('Number of lines: {}'.format(len(plays)))
word_count_play = [len(play.split()) for play in plays]
print('Average number of words in each line: {}'.format(np.average(word_count_play)))

print()
print('The plays {} to {}:'.format(*view_play_range))
print('\n'.join(text.split('\n')[view_play_range[0]:view_play_range[1]]))


Dataset Stats
Roughly the number of unique words: 35769
Number of games: 228
Average number of games in each game: 462.4078947368421
Number of lines: 105657
Average number of words in each line: 6.94456590666023

The plays 0 to 10:


GAME_START
,,,,,
Brook Lopez,Greg Monroe,Kyle Kuzma,Jump Ball Lopez vs. Monroe: Tip to Kuzma,,
Brandon Ingram,,,MISS Ingram 15' Jump Shot,,
Brook Lopez,,,Lopez REBOUND (Off:1 Def:0),,
Brook Lopez,,,Lopez 2' Layup (2 PTS),,
Tyler Ulis,,,,,MISS Ulis 26' 3PT Jump Shot
TJ Warren,,,,,Warren REBOUND (Off:1 Def:0)


### Create lookup table

In [105]:
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    # First version
    # word_counts = Counter(text)
    # sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    # int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    # vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}
    
    # Second version
    vocab = set(text)
    vocab_to_int = {c: i for i, c in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    return vocab_to_int, int_to_vocab

In [106]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token = dict()
    token['.'] = "||Period||"
    token[','] = "||Comma||"
    token['"'] = "||Quotation_Mark||"
    token[';'] = "||Semicolon||"
    token['!'] = "||Exclamation_Mark||"
    token['?'] = "||Question_Mark||"
    token['('] = "||Left_Parentheses||"
    token[')'] = "||Right_Parentheses||"
    token['--'] = "||Dash||"
    token['\n'] = "||Return||"

    return token

In [107]:
token_dict = token_lookup()
for key, token in token_dict.items():
    text = text.replace(key, ' {} '.format(token))

text = text.lower()
text = text.split()

vocab_to_int, int_to_vocab = create_lookup_tables(text)
int_text = [vocab_to_int[word] for word in text]
pickle.dump((int_text, vocab_to_int, int_to_vocab, token_dict), open('data/preprocess.p', 'wb'))


### Checking Tensorflow

In [108]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


/Users/ccuulinay/.pyenv/versions/3.6.2/envs/tf1_1/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: No GPU found. Please use a GPU to train your neural network.
  # This is added back by InteractiveShellApp.init_path()


In [109]:
# Get inputs place holder
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='target')
    learning_rate = tf.placeholder(tf.float32, name='learnning_rate')
    return inputs, targets, learning_rate




In [110]:
# Build lstm cell and initial state
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm_layers = 2
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    # cell = tf.contrib.rnn.MultiRNNCell([lstm] * lstm_layers)
    # cell = tf.contrib.rnn.MultiRNNCell([lstm for _ in range(lstm_layers)], state_is_tuple = True)
    def lstm_cell():
        return tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(lstm_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name='initial_state')
    return cell, initial_state

In [111]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1), name='embedding')
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed

In [112]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    # Seem no initial_state as param, then I added dtype param to make dynamic_rnn function works.
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name='final_state')
    print(final_state.get_shape().as_list())
    return outputs, final_state

In [113]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dimension):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    # embed_dimension = 300
    embed = get_embed(input_data, vocab_size, embed_dimension)
    print(embed.get_shape().as_list())
    outputs, final_state = build_rnn(cell, embed)
    print(final_state.get_shape().as_list())
    # Set activation_fn to None to keep the linear activation.
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None, 
                                               weights_initializer = tf.truncated_normal_initializer(stddev=0.1),
                                               biases_initializer=tf.zeros_initializer())
    
    return logits, final_state

In [114]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    n_batches = int(len(int_text) / (batch_size * seq_length))

    # Drop the last few characters to make only full batches
    xdata = np.array(int_text[: n_batches * batch_size * seq_length])
    ydata = np.array(int_text[1: n_batches * batch_size * seq_length + 1])
    ydata[-1] = xdata[0]

    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)
    # print(np.shape(list(zip(x_batches, y_batches))))
    return np.array(list(zip(x_batches, y_batches)))

In [125]:
# Number of Epochs
num_epochs = 30
# Batch Size
# batch_size = 5
batch_size = 256
# RNN Size
rnn_size = 512
# Sequence Length
# seq_length = 100
seq_length = 15
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 25
embed_dim = 300


In [126]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    train_op = optimizer.apply_gradients(capped_gradients)

[None, None, 300]
[2, 2, None, 512]
[2, 2, None, 512]


In [127]:
batches = get_batches(int_text, batch_size, seq_length)
save_dir = './save'
time_str = datetime.datetime.today().strftime('%Y%m%d%H%M')
# save_dir = './save_model_' + str(time_str)
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/459   train_loss = 7.230
Epoch   0 Batch   25/459   train_loss = 3.737
Epoch   0 Batch   50/459   train_loss = 2.869
Epoch   0 Batch   75/459   train_loss = 2.392
Epoch   0 Batch  100/459   train_loss = 2.248
Epoch   0 Batch  125/459   train_loss = 2.006
Epoch   0 Batch  150/459   train_loss = 1.903
Epoch   0 Batch  175/459   train_loss = 1.834
Epoch   0 Batch  200/459   train_loss = 1.712
Epoch   0 Batch  225/459   train_loss = 1.539
Epoch   0 Batch  250/459   train_loss = 1.549
Epoch   0 Batch  275/459   train_loss = 1.483
Epoch   0 Batch  300/459   train_loss = 1.475
Epoch   0 Batch  325/459   train_loss = 1.392
Epoch   0 Batch  350/459   train_loss = 1.424
Epoch   0 Batch  375/459   train_loss = 1.377
Epoch   0 Batch  400/459   train_loss = 1.342
Epoch   0 Batch  425/459   train_loss = 1.304
Epoch   0 Batch  450/459   train_loss = 1.313
Epoch   1 Batch   16/459   train_loss = 1.230
Epoch   1 Batch   41/459   train_loss = 1.277
Epoch   1 Batch   66/459   train_l

In [129]:
# Save seq_length and save_dir
pickle.dump((seq_length, save_dir), open('data/seq_len_n_save_dir.p', 'wb'))

### Checkpoint

In [130]:
import tensorflow as tf
import numpy as np

_, vocab_to_int, int_to_vocab, token_dict = pickle.load(open('data/preprocess.p', mode='rb'))
seq_length, load_dir = pickle.load(open('data/seq_len_n_save_dir.p', mode='rb'))

In [131]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name('input:0')
    InitialStateTensor = loaded_graph.get_tensor_by_name('initial_state:0')
    FinalStateTensor = loaded_graph.get_tensor_by_name('final_state:0')
    ProbsTensor = loaded_graph.get_tensor_by_name('probs:0')
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor




In [132]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    # print(probabilities)
    # print(int_to_vocab)
    p = np.squeeze(probabilities)
    
    # index = np.argsort(p)[-1:][0]
    # word = int_to_vocab[index]
    
    p[np.argsort(p)[:-5]] = 0
    p = p / np.sum(p)
    index = np.random.choice(len(int_to_vocab), 1, p=p)[0]
    word = int_to_vocab[index]
    return word

In [135]:
gen_length = 200
prime_word = 'game_start'


loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    # gen_sentences = [prime_word + ':']
    gen_sentences = [prime_word]
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save


INFO:tensorflow:Restoring parameters from ./save


game_start
,,,,,
nerlens noel, anthony davis, malcolm delaney, jump ball noel vs. tucker: tip to,,
marco belinelli, dennis schroder,, belinelli 19' pullup jump shot(2 pts)(prince 2 ast)
,,, cavaliers timeout: regular(full 4 short 0),,
,,,,,
og anunoby, jonas valanciunas,,,, anunoby s. foul(p1. t2)(h. workman),,
,,,,, raptors timeout: regular(reg. 2 short 0)
demar derozan,,,,, derozan free throw 1 of 2(9 pts)
,,,,, wizards timeout: regular(reg. 5 short 0)
justise winslow, wayne ellington,,,, sub: ellington for winslow
james johnson,,, johnson free throw 1 of 2(4 pts),,
josh jackson,,, miss jackson free throw 2 of 2,,
wayne ellington,,,,,


'201711192107'